In [1]:
import numpy as np
import torch
import torch.nn as nn
import torch.nn.functional
from flask import Flask, request, jsonify,send_from_directory
import json
#import onnxruntime

In [2]:
# demo code 
app = Flask(__name__)

# global variables
global train_logs 
softmax = nn.Softmax(dim=0)

@app.route('/')
def base():
    return "Base Test is OK" # may return a html file

In [3]:
def encodeLabel(onehats,dim):
    labelcoding = []
    for code in onehats:
        value = 0
        for idx in range(dim):
            if(code[idx]==1):
                value = idx
        labelcoding.append(value)
    labelcoding = torch.FloatTensor(labelcoding) 
    return labelcoding  

# Modle 1

In [4]:
# interface for users to upload dataset
@app.route('/dataset_1', methods=["POST"])
def getDataSet_1():
    global inputs_1
    global labels_1
    request_content = request.get_json()    
    #print(type(request_content))
    
    if(isinstance(request_content, str)):
        request_content = json.loads(request_content)
        #print(type(request_content))
    
    inputs_1 = request_content["inputs"]
    labels_1 = request_content["labels"]
    
    inputs_1 = torch.FloatTensor(inputs_1)
    labels_1 = torch.FloatTensor(labels_1)    
    
    response = app.response_class(
        response=json.dumps(request_content),
        status=200,
        mimetype='application/json'
    )
        
    return response
    
# if dataset is meta format, like image, json, yaml, ...
#   students may implement the approach to save the file or store data into an NoSQL database

In [5]:
# create model_1

# Make device agnostic code
device = "cpu"# Enable CUDA if possible
print(f"device is {device}")

class MyModel(nn.Module):
    def __init__(self, input_dim, output_dim):
        super(MyModel, self).__init__()
        
        hidden1_dim = 20
        hidden2_dim = 20

        self.linear_1 = nn.Linear(input_dim, hidden1_dim)
        self.linear_2 = nn.Linear(hidden1_dim, hidden2_dim)
        self.linear_3 = nn.Linear(hidden2_dim, output_dim)

        self.LogSoftmax = nn.LogSoftmax(dim=0)
        self.relu = nn.ReLU()

    def forward(self, inputs):
        outputs = self.linear_1(inputs).requires_grad_()
        outputs = self.LogSoftmax(outputs).requires_grad_()
        outputs = self.linear_2(outputs).requires_grad_()
        outputs = self.LogSoftmax(outputs).requires_grad_()        
        outputs = self.linear_3(outputs).requires_grad_()
        outputs = self.LogSoftmax(outputs).requires_grad_()

        return outputs
        
# 4. Create an instance of the model and send it to target device
torch.manual_seed(42)
model_1 = MyModel(7,2).to(device)

print(model_1.parameters)# Type of parameter object

device is cpu
<bound method Module.parameters of MyModel(
  (linear_1): Linear(in_features=7, out_features=20, bias=True)
  (linear_2): Linear(in_features=20, out_features=20, bias=True)
  (linear_3): Linear(in_features=20, out_features=2, bias=True)
  (LogSoftmax): LogSoftmax(dim=0)
  (relu): ReLU()
)>


In [6]:
# Create a loss function
criterion = nn.CrossEntropyLoss()  

# Create an optimizer
learning_rate = 0.0001
optimizer_1 = torch.optim.SGD(params=model_1.parameters(),lr=learning_rate)

@app.route("/train_1", methods=['GET', 'POST'])
def trainModel_1():
    global train_logs_1
    global inputs_1
    global labels_1
    
    train_logs_1 = []    
    num_epochs = 1000 # Train for longer\
    learning_rate = 0.01
    softmax = nn.Softmax(dim=0)
    
    # Put data to target device
    # device = "cpu" if torch.cuda.is_available() else "cpu"# Enable CUDA if possible
    device = "cpu"
    print(f"device is {device}")
    
    model_1.to(device)
    
    labels_1 = encodeLabel(labels_1,2)
    inputs_1, labels_1 = inputs_1.to(device), labels_1.to(device)

    for epoch in range(num_epochs):
        predicts = model_1(inputs_1)
        loss = criterion(predicts,labels_1.long() )  # https://www.cnblogs.com/blogwangwang/p/12018897.html
        optimizer_1.zero_grad()
        loss.backward()
        optimizer_1.step()

    # Print out what's happening every 10 epochs
        if epoch % 100 == 0:
            train_logs_1.append(f"Epoch: {epoch} | Loss: {loss:.5f}")
    
    return "OK"

In [7]:
@app.route("/log_1", methods=['GET'])
def getTrainingLog_1():    
    response = app.response_class(
        response=json.dumps(train_logs_1),
        status=200,
        mimetype='application/json'
    )
    return response

In [8]:
@app.route("/inference_1", methods=['POST'])
def inference_1():
    device = "cpu"
    request_content = request.get_json() 
    if(isinstance(request_content, str)):
        request_content = json.loads(request_content)
    input_data = request_content["input"]
    
    print(input_data)
    
    input_data = torch.FloatTensor(input_data)
    input_data = input_data.to(device)
    result = softmax(model_1(input_data)).tolist()
    
    response = app.response_class(
        response=json.dumps(result),
        status=200,
        mimetype='application/json'
    ) 
    return response

# Model 2

In [9]:
# interface for users to upload dataset
@app.route('/dataset_2', methods=["POST"])
def getDataSet():
    global inputs_2
    global labels_2 
    request_content = request.get_json()    
    #print(type(request_content))
    
    if(isinstance(request_content, str)):
        request_content = json.loads(request_content)
        #print(type(request_content))
    
    inputs_2 = request_content["inputs"]
    labels_2 = request_content["labels"]
    
    inputs_2 = torch.FloatTensor(inputs_2)
    labels_2 = torch.FloatTensor(labels_2)    
    
    response = app.response_class(
        response=json.dumps(request_content),
        status=200,
        mimetype='application/json'
    )
        
    return response
    
# if dataset is meta format, like image, json, yaml, ...
#   students may implement the approach to save the file or store data into an NoSQL database

In [10]:
# create model 2

# Make device agnostic code
device = "cpu"# Enable CUDA if possible
print(f"device is {device}")

torch.manual_seed(42)
model_2 = MyModel(7,2).to(device)

print(model_2.parameters)# Type of parameter object

device is cpu
<bound method Module.parameters of MyModel(
  (linear_1): Linear(in_features=7, out_features=20, bias=True)
  (linear_2): Linear(in_features=20, out_features=20, bias=True)
  (linear_3): Linear(in_features=20, out_features=2, bias=True)
  (LogSoftmax): LogSoftmax(dim=0)
  (relu): ReLU()
)>


In [11]:
# Create a loss function
criterion = nn.CrossEntropyLoss()  

# Create an optimizer
learning_rate = 0.0001
optimizer_2 = torch.optim.SGD(params=model_2.parameters(),lr=learning_rate)

@app.route("/train_2", methods=['GET', 'POST'])
def trainModel_2():
    global train_logs_2
    global inputs_2
    global labels_2
    
    train_logs_2 = []    
    num_epochs = 3000 # Train for longer\
    learning_rate = 0.01
    softmax = nn.Softmax(dim=0)
    
    # Put data to target device
    # device = "cpu" if torch.cuda.is_available() else "cpu"# Enable CUDA if possible
    device = "cpu"
    print(f"device is {device}")
    
    model_2.to(device)
    
    labels_2=encodeLabel(labels_2,2)
    inputs_2, labels_2 = inputs_2.to(device), labels_2.to(device)

    for epoch in range(num_epochs):
        predicts = model_2(inputs_2)
        loss = criterion(predicts,labels_2.long() )  # https://www.cnblogs.com/blogwangwang/p/12018897.html
        optimizer_2.zero_grad()
        loss.backward()
        optimizer_2.step()

    # Print out what's happening every 10 epochs
        if epoch % 100 == 0:
            train_logs_2.append(f"Epoch: {epoch} | Loss: {loss:.5f}")
    
    return "OK"

In [12]:
@app.route("/trainedModel_2", methods=['GET'])
def getTrainedModel_2():
    x = [1,2,3]
    x = torch.FloatTensor(x).to(device)
    model_2.eval()
    torch.onnx.export(model_2,x,"trainedModel.onnx")
    
    try:
        return send_from_directory('' , 'trainedModel.onnx', as_attachment=True)
    except FileNotFoundError:
        abort(404)

In [13]:
@app.route("/log_2", methods=['GET'])
def getTrainingLog_2():    
    response = app.response_class(
        response=json.dumps(train_logs_2),
        status=200,
        mimetype='application/json'
    )
    return response

In [14]:
@app.route("/inference_2", methods=['POST'])
def inference_2():
    device = "cpu"
    request_content = request.get_json() 
    if(isinstance(request_content, str)):
        request_content = json.loads(request_content)
    input_data = request_content["input"]
    
    input_data = torch.FloatTensor(input_data)
    input_data = input_data.to(device)
    result = softmax(model_2(input_data)).tolist()
    
    response = app.response_class(
        response=json.dumps(result),
        status=200,
        mimetype='application/json'
    ) 
    return response

In [ ]:
# Run up your Web API Server
if __name__ == '__main__':
    app.run(host='0.0.0.0',port=5000)

 * Serving Flask app '__main__' (lazy loading)
 * Environment: production
   Use a production WSGI server instead.
 * Debug mode: off


 * Running on all addresses.
 * Running on http://172.17.0.17:5000/ (Press CTRL+C to quit)
192.168.2.102 - - [26/Dec/2023 11:16:03] "GET / HTTP/1.1" 200 -
192.168.2.102 - - [26/Dec/2023 11:16:03] "POST /dataset_2 HTTP/1.1" 200 -


device is cpu


192.168.2.102 - - [26/Dec/2023 11:21:48] "GET /train_2 HTTP/1.1" 200 -
192.168.2.102 - - [26/Dec/2023 11:21:48] "GET /log_2 HTTP/1.1" 200 -
192.168.2.102 - - [26/Dec/2023 11:21:48] "POST /inference_2 HTTP/1.1" 200 -
